# Generate Timely and Effective Care dataframe

In [1]:
import pandas as pd
import numpy as np
import warnings
from IPython.utils import io
import sys
import time

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

main_dir = '~/Desktop/Rush/CMS_HospitalArchives/'

## Define Custom Functions

In [2]:
def curate(df):

    try:
        df = df[df['Facility ID'] != np.nan]
        df['Facility ID'] = df['Facility ID'].values.astype(str)
        
        ids = df['Facility ID'].tolist()
        ids2 = []
        for i in ids:
            if len(i) < 6:
                i = '0' + i
            ids2.append(i)
        df['Facility ID'] = ids2
        
    except:
        pass
    try:
        df = df[df['Facility Name'] != np.nan]
    except:
        pass
    
    for c in list(df):    
        try:
            df[c] = df[c].str.replace("\t","")
        except:
            pass

    return df


def rename_and_fill(df):
    
    cols1 = ['Provider ID', "Measure Start Date", "Measure End Date",
             'Hospital Name', 'Address 1', 'City/Town', 'County/Parish',
             'Telephone Number',
             ]
    
    
    cols2 = ['Facility ID', "Start Date", "End Date",
             'Facility Name', 'Address', 'City', 'County Name',
             'Phone Number',
             ]
    
    for i, col in enumerate(cols1):
        if col in list(df):
            df.rename(columns={col: cols2[i]}, inplace=True)
            
            
    cols = [
        ]
    
    for col in cols:
        if col not in list(df):
            df[col] = float('NaN')
    
    l = list(df)
    l = list(set([x for x in l if l.count(x) > 1]))
    if len(l) > 0:
        print('duplicates:', l)
        sys.exit()
        
    return df


def process2(df, lists, yr, mo):
    df = rename_and_fill(df)
    df = curate(df)
    lists.append(list(df))
    df['file_month'] = [mo]* df.shape[0]
    df['file_year'] = [yr]* df.shape[0]
    df = df.reindex(sorted(df.columns), axis=1)
    return df, lists

## Load Files

In [3]:
df_list = []
lists = []

yrs = ['2023', '2023', '2023', '2023',
       '2022', '2022', '2022', '2022',
       '2021','2021','2021', '2021', '2021',
       '2020', '2020', '2020', '2020', 
       '2019', '2019', '2019', '2019', 
       '2018', '2018', '2018', '2018',
       '2017', '2017', '2017',
       '2016', '2016', '2016', '2016',
       '2015', '2015', '2015', '2015', '2015', '2015',
       '2014', '2014', '2014',
       ]

mos = ['01', '04', '07', '10',
       '01', '04', '07', '10',
       '01', '03', '04', '07', '10', 
       '10', '07', '04', '01', 
       '10', '07', '04', '03', 
       '10', '07', '05', '01',
       '10', '07', '04',
       '12', '11', '08', '05',
       '12', '10', '07', '05', '04', '01',
       '12', '10', '07',
       ]

subdirs = ['2023/hospitals_01_2023/Timely_and_Effective_Care-Hospital.csv', 
           '2023/hospitals_04_2023/Timely_and_Effective_Care-Hospital.csv',
           '2023/hospitals_07_2023/Timely_and_Effective_Care-Hospital.csv',
           '2023/hospitals_10_2023/Timely_and_Effective_Care-Hospital.csv',
           
           '2022/hospitals_01_2022/Timely_and_Effective_Care-Hospital.csv', 
           '2022/hospitals_04_2022/Timely_and_Effective_Care-Hospital.csv',
           '2022/hospitals_07_2022/Timely_and_Effective_Care-Hospital.csv',
           '2022/hospitals_10_2022/Timely_and_Effective_Care-Hospital.csv',
           
           '2021/hospitals_01_2021/Timely_and_Effective_Care-Hospital.csv',
           '2021/hospitals_03_2021/Timely_and_Effective_Care-Hospital.csv',
           '2021/hospitals_04_2021/Timely_and_Effective_Care-Hospital.csv',
           '2021/hospitals_07_2021/Timely_and_Effective_Care-Hospital.csv',
           '2021/hospitals_10_2021/Timely_and_Effective_Care-Hospital.csv',
           
           '2020/hospitals_archive_10_2020/Timely_and_Effective_Care_Hospital.csv',
           '2020/hospitals_archive_07_2020/Timely_and_Effective_Care_Hospital.csv',
           '2020/HOSArchive_Revised_Flatfiles_20200422/Timely and Effective Care - Hospital.csv',
           '2020/HOSArchive_Revised_Flatfiles_20200129/Timely and Effective Care - Hospital.csv',
           
           '2019/HOSArchive_Revised_Flatfiles_20191030/Timely and Effective Care - Hospital.csv',
           '2019/HOSArchive_Revised_Flatfiles_20190702/Timely and Effective Care - Hospital.csv',
           '2019/HOSArchive_Revised_FlatFiles_20190424/Timely and Effective Care - Hospital.csv',
           '2019/HOSArchive_Revised_Flatfiles_20190321/Timely and Effective Care - Hospital.csv',
           
           '2018/HOSArchive_Revised_FlatFiles_20181031/Timely and Effective Care - Hospital.csv',
           '2018/HOSArchive_Revised_FlatFiles_20180725/Timely and Effective Care - Hospital.csv', 
           '2018/HOSArchive_Revised_FlatFiles_20180523/Timely and Effective Care - Hospital.csv',
           '2018/HOSArchive_Revised_FlatFiles_20180126/Timely and Effective Care - Hospital.csv',
           
           '2017/HOSArchive_Revised_FlatFiles_20171024/Timely and Effective Care - Hospital.csv',
           '2017/HOSArchive_Revised_FlatFiles_20170726/Timely and Effective Care - Hospital.csv', 
           '2017/HOSArchive_Revised_Flatfiles_20170428/Timely and Effective Care - Hospital.csv', 
           
           '2016/HOSArchive_Revised_Flatfiles_20161219/Timely and Effective Care - Hospital.csv',
           '2016/Hospital_Revised_FlatFiles_20161110/Timely and Effective Care - Hospital.csv', 
           '2016/HOSArchive_Revised_FlatFiles_20160810/Timely and Effective Care - Hospital.csv',
           '2016/HOSArchive_Revised_FlatFiles_20160504/Timely and Effective Care - Hospital.csv',
           
           '2015/HOSArchive_Revised_FlatFiles_20151210/Timely and Effective Care - Hospital.csv',
           '2015/HOSArchive_Revised_FlatFiles_20151008/Timely and Effective Care - Hospital.csv',
           '2015/HOSArchive_Revised_FlatFiles_20150716/Timely and Effective Care - Hospital.csv',
           '2015/HOSArchive_Revised_Flatfiles_20150506/Timely and Effective Care - Hospital.csv',
           '2015/HOSArchive_Revised_Flatfiles_20150416/Timely and Effective Care - Hospital.csv',
           '2015/HOSArchive_Revised_Flatfiles_20150122/Timely and Effective Care - Hospital.csv',
           
           '2014/HOSArchive_Revised_Flatfiles_20141218/Timely and Effective Care - Hospital.csv',
           '2014/HOSArchive_Revised_Flatfiles_20141023/Timely and Effective Care - Hospital.csv',
           '2014/HOSArchive_Revised_Flatfiles_20140717/Timely and Effective Care - Hospital.csv',
           ]


for i, subdir in enumerate(subdirs):
    with io.capture_output() as captured: df = pd.read_csv(main_dir + subdir, encoding = "ISO-8859-1")
    print(subdir)# + ' :  (rows, columns) =', df.shape)
    df, lists = process2(df, lists, yrs[i], mos[i])
    df_list.append(df)

subdir = 'Timely_and_Effective_Care/CombinedFiles_Timely_and_Effective_Care'
df = pd.concat(df_list)

print('df.shape:', df.shape)
df = df[~df['Score'].isin([np.nan, float("NaN"), 'Not Available'])]
print('df.shape:', df.shape)

print(sorted(list(df)))
df.drop(labels=['Address', 'City', 'County Name', 'Footnote', 'Phone Number', 'State', 'ZIP Code'], axis=1, inplace=True)
df.head()

2023/hospitals_01_2023/Timely_and_Effective_Care-Hospital.csv
2023/hospitals_04_2023/Timely_and_Effective_Care-Hospital.csv
2023/hospitals_07_2023/Timely_and_Effective_Care-Hospital.csv
2023/hospitals_10_2023/Timely_and_Effective_Care-Hospital.csv
2022/hospitals_01_2022/Timely_and_Effective_Care-Hospital.csv
2022/hospitals_04_2022/Timely_and_Effective_Care-Hospital.csv
2022/hospitals_07_2022/Timely_and_Effective_Care-Hospital.csv
2022/hospitals_10_2022/Timely_and_Effective_Care-Hospital.csv
2021/hospitals_01_2021/Timely_and_Effective_Care-Hospital.csv
2021/hospitals_03_2021/Timely_and_Effective_Care-Hospital.csv
2021/hospitals_04_2021/Timely_and_Effective_Care-Hospital.csv
2021/hospitals_07_2021/Timely_and_Effective_Care-Hospital.csv
2021/hospitals_10_2021/Timely_and_Effective_Care-Hospital.csv
2020/hospitals_archive_10_2020/Timely_and_Effective_Care_Hospital.csv
2020/hospitals_archive_07_2020/Timely_and_Effective_Care_Hospital.csv
2020/HOSArchive_Revised_Flatfiles_20200422/Timely and 

,Condition,End Date,Facility ID,Facility Name,Measure ID,Measure Name,Sample,Score,Start Date,file_month,file_year
0,Emergency Department,12/31/2021,010001,SOUTHEAST HEALTH MEDICAL CENTER,EDV,Emergency department volume,NaN,high,01/01/2021,01,2023
3,Healthcare Personnel Vaccination,03/31/2022,010001,SOUTHEAST HEALTH MEDICAL CENTER,HCP_COVID_19,Percentage of healthcare personnel who completed COVID-19 primary vaccination series,2323,73.7,01/01/2022,01,2023
4,Healthcare Personnel Vaccination,03/31/2022,010001,SOUTHEAST HEALTH MEDICAL CENTER,IMM_3,Healthcare workers given influenza vaccination,3795,97,10/01/2021,01,2023
5,Emergency Department,03/31/2022,010001,SOUTHEAST HEALTH MEDICAL CENTER,OP_18b,Average (median) time patients spent in the emergency department before leaving from the visit A lower number of minutes is better,323,205,04/01/2021,01,2023
6,Emergency Department,03/31/2022,010001,SOUTHEAST HEALTH MEDICAL CENTER,OP_18c,Average (median) time patients spent in the emergency department before leaving from the visit- Psychiatric/Mental Health Patients. A lower number of minutes is better,14,199,04/01/2021,01,2023


In [4]:
labs = ['Measure ID', 'Measure Name', 'Sample', 'Score', 'Condition']

for l in labs:
    print(l)
    labs2 = []
    vals = df[l].tolist()
    
    for v in vals:
        try:
            v = v.strip()
        except:
            pass
        
        try:
            if "\x93" in v:
                v = v.replace("\x93", '')
            if "\x94" in v:
                v = v.replace("\x94", '')
            if "\x96" in v:
                v = v.replace("\x96", '')
            if '"' in v:
                v = v.replace('"', '')
            if "'" in v:
                v = v.replace("'", '')
            if "  " in v:
                v = v.replace("  ", ' ')
            if "  " in v:
                v = v.replace("  ", ' ')    
        except:
            pass

        labs2.append(v)
    df[l] = labs2


Measure ID
Measure Name
Sample
Score
Condition


In [5]:
df = df.filter(items=['Facility ID', 'Facility Name', 'file_month', 'file_year', 'Condition',
                      'Measure ID', 'Measure Name', 'Start Date', 'End Date',
                      'Score', 'Sample'], axis=1)

ids = df['Measure ID'].tolist()
ids2 = []
for i in ids:
    
    if i == 'STK_02':
        i = 'STK_2'
            
    elif i == 'STK_03':
        i = 'STK_3'
        
    elif i == 'STK_05':
        i = 'STK_5'
        
    elif i == 'STK_06':
        i = 'STK_6'
        
    ids2.append(i)
    
df['Measure ID'] = ids2


In [6]:
n1 = df['Measure Name'].tolist()
n2 = []

for n in n1:
    n = n.strip()
    
    if n == 'Venous Thromboembolism (VTE) Prophylaxis' or n == 'Venous thromboembolism prophylaxis':
        n = 'Venous Thromboembolism Prophylaxis'

    elif n == 'Admit Decision Time to ED Departure Time for Admitted Patients psychiatric/mental health disorders':
        n = 'Admit Decision Time to ED Departure Time for Admitted Patients - psychiatric/mental health disorders'

    elif n == 'Average (median) time patients spent in the emergency department before leaving from the visit A lower number of minutes is better':
        n = 'Median time patients spent in the ED before leaving from visit'
        
    elif n == 'Average (median) time patients spent in the emergency department before leaving from the visit- Psychiatric/Mental Health Patients.  A lower number of minutes is better':
        n = 'Median time Psychiatric/Mental Health patients spent in the ED before leaving from visit'
    
    elif n == 'Average (median) time patients spent in the emergency department before leaving from the visit- Psychiatric/Mental Health Patients. A lower number of minutes is better':
        n = 'Median time Psychiatric/Mental Health patients spent in the ED before leaving from visit'
        
    elif n == 'Average (median) time patients spent in the emergency department, after the doctor decided to admit them as an inpatient before leaving the emergency department for their inpatient room A lower number of minutes is better':
        n = 'Median time patients spent in the ED after a doctor admitted them and before leaving the ED for an inpatient room'
        
    elif n == 'Average (median) time patients spent in the emergency department, before they were admitted to the hospital as an inpatient A lower number of minutes is better':
        n = 'Median time patients spent in the ED before they were admitted as an inpatient'
        
    elif n == 'Blood Cultures Performed in the Emergency Department Prior to Initial Antibiotic Received in Hospital':
        n = 'Blood Cultures Performed in the ED Prior to Initial Antibiotic Received in Hospital'
    
    elif n == 'Cardiac Surgery Patients With Controlled 6 A.M. Postoperative Blood Glucose':
        n = 'Cardiac surgery patients with controlled 6 a.m. postoperative blood glucose'
    
    elif n == 'Endoscopy/Polyp Surveillance: Appropriate Follow-Up Interval for Normal Colonoscopy in Average Risk Patients':
        n = 'Endoscopy/polyp surveillance: appropriate follow-up interval for normal colonoscopy in average risk patients'
        
    elif n == 'Endoscopy/Polyp Surveillance: Colonoscopy Interval for Patients with a History of Adenomatous Polyps - Avoidance of Inappropriate Use':
        n = 'Endoscopy/polyp surveillance: colonoscopy interval for patients with a history of adenomatous polyps - avoidance of inappropriate use'
        
    elif n == 'Hospital Acquired Potentially-Preventable Venous Thromboembolism':
        n = 'Hospital acquired potentially preventable venous thromboembolism'
        
    elif n == "Improvement in Patient's Visual Function within 90 Days Following Cataract Surgery":
        n = "Improvement in Patients Visual Function within 90 Days Following Cataract Surgery"
        
    elif n == "Improvement in Patients Visual Function within 90 Days Following Cataract Surgery":
        n = "Improvement in Patients Visual Function within 90 Days Following Cataract Surgery"
     
    elif n == 'OP 18':
        n = 'Median time patients spent in the ED before leaving from visit'
        
    elif n == 'Safe Use of Opioids  Concurrent Prescribing':
        n = 'Safe Use of Opioids - Concurrent Prescribing'
        
    elif n == 'Heart attack patients given drugs to break up blood clots within 30 minutes of arrival':
        n = 'Fibrinolytic Therapy Received Within 30 Minutes Of Hospital Arrival'
    
    elif n == 'ED1':
        n = 'Median time patients spent in the ED before they were admitted as an inpatient'
    
    elif n == 'ED2':
        n = 'Median time patients spent in the ED after a doctor admitted them and before leaving the ED for an inpatient room'
    
    elif n == 'OP-18':
        n = 'Median time Psychiatric/Mental Health patients spent in the ED before leaving from visit'
        
    elif n == 'Elective Delivery':
        n = 'Percent of newborns whose deliveries were scheduled 1-3 weeks early when a scheduled delivery was not medically necessary'
    
    elif n == 'Percent of newborns whose deliveries were scheduled early (1-3 weeks early), when a scheduled delivery was not medically necessary':
        n = 'Percent of newborns whose deliveries were scheduled 1-3 weeks early when a scheduled delivery was not medically necessary'
    
    elif n == 'Prophylactic antibiotic received within 1 hour prior to surgical incision':
        n = 'Prophylactic Antibiotic Initiated Within One Hour Prior to Surgical Incision'
    
    elif n == 'Safe Use of Opioids Concurrent Prescribing':
        n = 'Safe Use of Opioids - Concurrent Prescribing'

    n2.append(n)

df['Measure Name'] = n2

print('\n')
names = sorted(df['Measure Name'].unique())

df.head()

,Facility ID,Facility Name,file_month,file_year,Condition,Measure ID,Measure Name,Start Date,End Date,Score,Sample
0,010001,SOUTHEAST HEALTH MEDICAL CENTER,01,2023,Emergency Department,EDV,Emergency department volume,01/01/2021,12/31/2021,high,NaN
3,010001,SOUTHEAST HEALTH MEDICAL CENTER,01,2023,Healthcare Personnel Vaccination,HCP_COVID_19,Percentage of healthcare personnel who completed COVID-19 primary vaccination series,01/01/2022,03/31/2022,73.7,2323
4,010001,SOUTHEAST HEALTH MEDICAL CENTER,01,2023,Healthcare Personnel Vaccination,IMM_3,Healthcare workers given influenza vaccination,10/01/2021,03/31/2022,97,3795
5,010001,SOUTHEAST HEALTH MEDICAL CENTER,01,2023,Emergency Department,OP_18b,Median time patients spent in the ED before leaving from visit,04/01/2021,03/31/2022,205,323
6,010001,SOUTHEAST HEALTH MEDICAL CENTER,01,2023,Emergency Department,OP_18c,Median time Psychiatric/Mental Health patients spent in the ED before leaving from visit,04/01/2021,03/31/2022,199,14


In [7]:
df['Measure Name'] = df['Measure Name'] + ' (' + df['Measure ID'] + ' - ' + df['Condition'] + ')'

tdf = df.filter(items=['Measure Name'])
tdf.drop_duplicates(inplace=True)

print(tdf.shape)
print(len(df['Measure Name'].unique()), 'Measure Name, ID, Condition combinations')

tdf.sort_values(by=['Measure Name'], ascending=True, inplace=True)
tdf.head(100)

(77, 1)
77 Measure Name, ID, Condition combinations


,Measure Name
9,ACEI or ARB for LVSD (HF_3 - Heart Failure)
26,Admit Decision Time to ED Departure Time for Admitted Patients - non psychiatric/mental health disorders (ED_2_Strata_1 - Electronic Clinical Quality Measure)
27,Admit Decision Time to ED Departure Time for Admitted Patients - psychiatric/mental health disorders (ED_2_Strata_2 - Electronic Clinical Quality Measure)
320,Anticoagulation Therapy for Atrial Fibrillation/Flutter (STK_3 - Electronic Clinical Quality Measure)
75,Anticoagulation Therapy for Atrial Fibrillation/Flutter (STK_3 - Stroke Care)
39,Anticoagulation overlap therapy (VTE_3 - Blood Clot Prevention and Treatment)
46,Antithrombotic Therapy by End of Hospital Day 2 (STK_5 - Electronic Clinical Quality Measure)
34,Antithrombotic Therapy by End of Hospital Day 2 (STK_5 - Stroke Care)
14,Appropriate care for severe sepsis and septic shock (SEP_1 - Sepsis Care)
18,Aspirin at Arrival (OP_4 - Heart Attack or Chest Pain)


## Save subset of dataframe for use in the app

In [8]:
ls = ['Facility ID', 'Facility Name', 'file_month', 'file_year', 'Measure Name', 
      'Score', 'Sample', 'Start Date', 'End Date',]
cols = ['Facility ID', 'Facility Name', 'file_month', 'file_year', 'Start Date', 'End Date']
tdf = df.filter(items=ls, axis=1)

main_df = pd.DataFrame(columns=cols)
Measures = sorted(tdf['Measure Name'].unique())
for i, mi in enumerate(Measures):
    ttdf = tdf[tdf['Measure Name'] == mi]
    measures = sorted(ttdf['Measure Name'].unique())
    df2 = pd.DataFrame(columns=cols)

    for j, m in enumerate(measures):
        tdf2 = ttdf[ttdf['Measure Name'] == m]
        for n in list(tdf2):
            if n == 'Measure Name' or n in cols:
                continue
            else:
                tdf2[n] = pd.to_numeric(tdf2[n], errors='coerce')
                tdf2.rename(columns={n: m + ' (' + n + ')'}, inplace=True)
        
        tdf2.drop(labels=['Measure Name'], axis=1, inplace=True)
        
        df2 = df2.merge(tdf2, on=cols, how='outer')
    
    main_df = main_df.merge(df2, on=cols, how='outer')


tdf = main_df.copy(deep=True)
del df2, main_df

print(tdf.shape)
tdf = tdf.loc[:, ~tdf.T.duplicated(keep='first')]
tdf.dropna(how='all', axis=1, inplace=True)
print(tdf.shape)
tdf.drop_duplicates(inplace=True)
print(tdf.shape)
tdf.drop_duplicates(subset = cols, inplace=True)
print(tdf.shape)
tdf.head()

(540529, 160)
(540529, 158)
(540529, 158)
(540529, 158)


,Facility ID,Facility Name,file_month,file_year,ACEI or ARB for LVSD (HF_3 - Heart Failure) (Score),ACEI or ARB for LVSD (HF_3 - Heart Failure) (Sample),Start Date,End Date,Admit Decision Time to ED Departure Time for Admitted Patients - non psychiatric/mental health disorders (ED_2_Strata_1 - Electronic Clinical Quality Measure) (Score),Admit Decision Time to ED Departure Time for Admitted Patients - non psychiatric/mental health disorders (ED_2_Strata_1 - Electronic Clinical Quality Measure) (Sample),Admit Decision Time to ED Departure Time for Admitted Patients - psychiatric/mental health disorders (ED_2_Strata_2 - Electronic Clinical Quality Measure) (Score),Admit Decision Time to ED Departure Time for Admitted Patients - psychiatric/mental health disorders (ED_2_Strata_2 - Electronic Clinical Quality Measure) (Sample),Anticoagulation Therapy for Atrial Fibrillation/Flutter (STK_3 - Electronic Clinical Quality Measure) (Score),Anticoagulation Therapy for Atrial Fibrillation/Flutter (STK_3 - Electronic Clinical Quality Measure) (Sample),Anticoagulation Therapy for Atrial Fibrillation/Flutter (STK_3 - Stroke Care) (Score),Anticoagulation Therapy for Atrial Fibrillation/Flutter (STK_3 - Stroke Care) (Sample),Anticoagulation overlap therapy (VTE_3 - Blood Clot Prevention and Treatment) (Score),Anticoagulation overlap therapy (VTE_3 - Blood Clot Prevention and Treatment) (Sample),Antithrombotic Therapy by End of Hospital Day 2 (STK_5 - Electronic Clinical Quality Measure) (Score),Antithrombotic Therapy by End of Hospital Day 2 (STK_5 - Electronic Clinical Quality Measure) (Sample),Antithrombotic Therapy by End of Hospital Day 2 (STK_5 - Stroke Care) (Score),Antithrombotic Therapy by End of Hospital Day 2 (STK_5 - Stroke Care) (Sample),Appropriate care for severe sepsis and septic shock (SEP_1 - Sepsis Care) (Score),Appropriate care for severe sepsis and septic shock (SEP_1 - Sepsis Care) (Sample),Aspirin at Arrival (OP_4 - Heart Attack or Chest Pain) (Score),Aspirin at Arrival (OP_4 - Heart Attack or Chest Pain) (Sample),Aspirin prescribed at discharge (AMI_2 - Heart Attack or Chest Pain) (Score),Aspirin prescribed at discharge (AMI_2 - Heart Attack or Chest Pain) (Sample),Assessed for Rehabilitation (STK_10 - Stroke Care) (Score),Assessed for Rehabilitation (STK_10 - Stroke Care) (Sample),Blood Cultures Performed in the ED Prior to Initial Antibiotic Received in Hospital (PN_3b - Pneumonia) (Score),Blood Cultures Performed in the ED Prior to Initial Antibiotic Received in Hospital (PN_3b - Pneumonia) (Sample),Cardiac surgery patients with controlled 6 a.m. postoperative blood glucose (SCIP_INF_4 - Surgical Care Improvement Project) (Score),Cardiac surgery patients with controlled 6 a.m. postoperative blood glucose (SCIP_INF_4 - Surgical Care Improvement Project) (Sample),Discharge instructions (HF_1 - Heart Failure) (Score),Discharge instructions (HF_1 - Heart Failure) (Sample),Discharged on Antithrombotic Therapy (STK_2 - Electronic Clinical Quality Measure) (Score),Discharged on Antithrombotic Therapy (STK_2 - Electronic Clinical Quality Measure) (Sample),Discharged on Antithrombotic Therapy (STK_2 - Stroke Care) (Score),Discharged on Antithrombotic Therapy (STK_2 - Stroke Care) (Sample),Discharged on Statin Medication (STK_6 - Electronic Clinical Quality Measure) (Score),Discharged on Statin Medication (STK_6 - Electronic Clinical Quality Measure) (Sample),Discharged on Statin Medication (STK_6 - Stroke Care) (Score),Discharged on Statin Medication (STK_6 - Stroke Care) (Sample),Door to diagnostic eval (OP_20 - Emergency Department) (Score),Door to diagnostic eval (OP_20 - Emergency Department) (Sample),Endoscopy/polyp surveillance: appropriate follow-up interval for normal colonoscopy in average risk patients (OP_29 - Colonoscopy care) (Score),Endoscopy/polyp surveillance: appropriate follow-up interval for normal colonoscopy in average risk patients (OP_29 - Colonoscopy care) (Sample),Endoscopy/polyp surveillance: colonos

In [9]:
ttdf = tdf.drop(labels=['Start Date', 'End Date'], axis=1)
ttdf.drop_duplicates(inplace=True)
print(ttdf.shape)


(540529, 156)


In [10]:

start_time = time.time()
ttdf['marker'] = ttdf['Facility ID'] + ' | ' + ttdf['Facility Name']  + ' | ' + ttdf['file_month'] + ' | ' + ttdf['file_year']
end_time = time.time()
print("Run time = {:.3f} seconds".format(end_time - start_time))
print(ttdf.shape)

start_time = time.time()
column = ttdf.pop('marker')
ttdf.insert(0, column.name, column)
end_time = time.time()
print("Run time = {:.3f} seconds".format(end_time - start_time))
print(ttdf.shape)

start_time = time.time()
ttdf.drop_duplicates(inplace=True)
end_time = time.time()
print("Run time = {:.3f} seconds".format(end_time - start_time))
print(ttdf.shape)

start_time = time.time()
collapsed_df = ttdf.groupby(ttdf.marker).apply(lambda group: group.ffill().bfill().head(1))
end_time = time.time()
print("Run time = {:.3f} seconds".format(end_time - start_time))
print(ttdf.shape)

start_time = time.time()
ttdf = collapsed_df.reset_index(drop=True)
end_time = time.time()
print("Run time = {:.3f} seconds".format(end_time - start_time))
print(ttdf.shape)

start_time = time.time()
ttdf.drop_duplicates(inplace=True)
end_time = time.time()
print("Run time = {:.3f} seconds".format(end_time - start_time))
print(ttdf.shape)

ttdf.head()

Run time = 0.376 seconds
(540529, 157)
Run time = 0.004 seconds
(540529, 157)
Run time = 2.873 seconds
(540529, 157)
Run time = 5051.964 seconds
(540529, 157)
Run time = 0.254 seconds
(182846, 157)
Run time = 0.948 seconds
(182846, 157)


,marker,Facility ID,Facility Name,file_month,file_year,ACEI or ARB for LVSD (HF_3 - Heart Failure) (Score),ACEI or ARB for LVSD (HF_3 - Heart Failure) (Sample),Admit Decision Time to ED Departure Time for Admitted Patients - non psychiatric/mental health disorders (ED_2_Strata_1 - Electronic Clinical Quality Measure) (Score),Admit Decision Time to ED Departure Time for Admitted Patients - non psychiatric/mental health disorders (ED_2_Strata_1 - Electronic Clinical Quality Measure) (Sample),Admit Decision Time to ED Departure Time for Admitted Patients - psychiatric/mental health disorders (ED_2_Strata_2 - Electronic Clinical Quality Measure) (Score),Admit Decision Time to ED Departure Time for Admitted Patients - psychiatric/mental health disorders (ED_2_Strata_2 - Electronic Clinical Quality Measure) (Sample),Anticoagulation Therapy for Atrial Fibrillation/Flutter (STK_3 - Electronic Clinical Quality Measure) (Score),Anticoagulation Therapy for Atrial Fibrillation/Flutter (STK_3 - Electronic Clinical Quality Measure) (Sample),Anticoagulation Therapy for Atrial Fibrillation/Flutter (STK_3 - Stroke Care) (Score),Anticoagulation Therapy for Atrial Fibrillation/Flutter (STK_3 - Stroke Care) (Sample),Anticoagulation overlap therapy (VTE_3 - Blood Clot Prevention and Treatment) (Score),Anticoagulation overlap therapy (VTE_3 - Blood Clot Prevention and Treatment) (Sample),Antithrombotic Therapy by End of Hospital Day 2 (STK_5 - Electronic Clinical Quality Measure) (Score),Antithrombotic Therapy by End of Hospital Day 2 (STK_5 - Electronic Clinical Quality Measure) (Sample),Antithrombotic Therapy by End of Hospital Day 2 (STK_5 - Stroke Care) (Score),Antithrombotic Therapy by End of Hospital Day 2 (STK_5 - Stroke Care) (Sample),Appropriate care for severe sepsis and septic shock (SEP_1 - Sepsis Care) (Score),Appropriate care for severe sepsis and septic shock (SEP_1 - Sepsis Care) (Sample),Aspirin at Arrival (OP_4 - Heart Attack or Chest Pain) (Score),Aspirin at Arrival (OP_4 - Heart Attack or Chest Pain) (Sample),Aspirin prescribed at discharge (AMI_2 - Heart Attack or Chest Pain) (Score),Aspirin prescribed at discharge (AMI_2 - Heart Attack or Chest Pain) (Sample),Assessed for Rehabilitation (STK_10 - Stroke Care) (Score),Assessed for Rehabilitation (STK_10 - Stroke Care) (Sample),Blood Cultures Performed in the ED Prior to Initial Antibiotic Received in Hospital (PN_3b - Pneumonia) (Score),Blood Cultures Performed in the ED Prior to Initial Antibiotic Received in Hospital (PN_3b - Pneumonia) (Sample),Cardiac surgery patients with controlled 6 a.m. postoperative blood glucose (SCIP_INF_4 - Surgical Care Improvement Project) (Score),Cardiac surgery patients with controlled 6 a.m. postoperative blood glucose (SCIP_INF_4 - Surgical Care Improvement Project) (Sample),Discharge instructions (HF_1 - Heart Failure) (Score),Discharge instructions (HF_1 - Heart Failure) (Sample),Discharged on Antithrombotic Therapy (STK_2 - Electronic Clinical Quality Measure) (Score),Discharged on Antithrombotic Therapy (STK_2 - Electronic Clinical Quality Measure) (Sample),Discharged on Antithrombotic Therapy (STK_2 - Stroke Care) (Score),Discharged on Antithrombotic Therapy (STK_2 - Stroke Care) (Sample),Discharged on Statin Medication (STK_6 - Electronic Clinical Quality Measure) (Score),Discharged on Statin Medication (STK_6 - Electronic Clinical Quality Measure) (Sample),Discharged on Statin Medication (STK_6 - Stroke Care) (Score),Discharged on Statin Medication (STK_6 - Stroke Care) (Sample),Door to diagnostic eval (OP_20 - Emergency Department) (Score),Door to diagnostic eval (OP_20 - Emergency Department) (Sample),Endoscopy/polyp surveillance: appropriate follow-up interval for normal colonoscopy in average risk patients (OP_29 - Colonoscopy care) (Score),Endoscopy/polyp surveillance: appropriate follow-up interval for normal colonoscopy in average risk patients (OP_29 - Colonoscopy care) (Sample),Endoscopy/polyp surveillance: colonoscopy interval

In [11]:
ttdf.drop(labels=['marker'], axis=1, inplace=True)
ttdf.to_pickle('~/GitHub/hospitals-data-archive/dataframes/partial_dataframes/Timely_and_Effective_Care_df.pkl.gz', protocol=5, compression='gzip')


In [12]:
m1 = list(ttdf)
ls = ['Facility ID','Facility Name','file_month','file_year']
for l in ls: 
    m1.remove(l)

## Save measurement dates

In [13]:
# Columns to keep as is
id_cols = ['Facility ID', 'Facility Name', 'file_month', 'file_year', 'Start Date', 'End Date']

# Melt the specific columns and create the 'Measure' and 'Score' columns
measures_df = tdf.melt(id_vars=id_cols, var_name='Measure Name', value_name='Score')
measures_df.drop(labels=['Score', 'Facility ID', 'Facility Name'], axis=1, inplace=True)

print(measures_df.shape)
measures_df.drop_duplicates(inplace=True)
measures_df.reset_index(drop=True, inplace=True)
print(measures_df.shape)

measures_df['Start Date'] = pd.to_datetime(measures_df['Start Date'])
measures_df['End Date'] = pd.to_datetime(measures_df['End Date'])
measures_df.to_csv('~/GitHub/hospitals-data-archive/measure_dates/Timely_and_Effective_Care_df.csv')

measures_df.head()

(82160408, 5)
(22040, 5)


,file_month,file_year,Start Date,End Date,Measure Name
0,10,2015,2014-01-01,2014-12-31,ACEI or ARB for LVSD (HF_3 - Heart Failure) (Score)
1,07,2015,2013-10-01,2014-09-30,ACEI or ARB for LVSD (HF_3 - Heart Failure) (Score)
2,05,2015,2013-07-01,2014-06-30,ACEI or ARB for LVSD (HF_3 - Heart Failure) (Score)
3,04,2015,2013-07-01,2014-06-30,ACEI or ARB for LVSD (HF_3 - Heart Failure) (Score)
4,01,2015,2013-04-01,2013-12-31,ACEI or ARB for LVSD (HF_3 - Heart Failure) (Score)


In [14]:
m2 = measures_df['Measure Name'].unique().tolist()
sorted(m1) == sorted(m2)

True